# Coding Confirmation to 일람표 Converter

**Coding Confimation.xlsx**
* 엑셀 sheet 이름: "AE" 

Cell을 순서대로 선택하고 (오른 마우스 클릭) 파랗게 선택된 상태에 cell을 실행하세요 `Shift` + `Enter`
* Cell 옆에 별표 `[*]`은 계산중이나 사용자 입력을 기다리는 뜻을 의미합니다.
* 파일 업로드 후, `(1)` 표시 확인.
* 파일을 다시 업로드하는 경우, cell을 다시 실행 하세요 (1. cell 선택 2. cell 실행 `Shift` + `Enter`)

In [1]:
import utility

uploaded = utility.prompt_upload()

FileUpload(value={}, description='Upload Coding Confirmation (.xlsx)', layout=Layout(width='270px'))

문자 처리하는 두 가지 모드가 있습니다

1. 삭제 모드: 사용자가 입력한 문자와 정확히 일치하는 행(또는 열) 삭제
    * 행/열 삭제는 상황에 따라서 자동으로 구분합니다.
2. 수정 모드: Excel 테이블의 값을 편집 (실수의 경우 등). 
    * E.g., ADR 열이 3개의 값이 있는 경우 `["ADR", "non-ADR", "non ADR"]`, 두가지 방법으로 처리할 수 있습니다. 
        * "non-ADR"을 "non ADR"로 수정 (하이픈 `-` 제거)
        * 반대로 "non ADR"을 "non-ADR"로 수정 (하이픈 `-` 추가)

에러나 다시 입력해야하는 경우의 패널 중지 (▣)을 누르시고, cell을 다시 실행하세요

In [ ]:
final = utility.control_process(uploaded[0], uploaded[1], option=0)
final

파일 내보내기: 
1. `to_excel("파일 이름")`함수 안에 원하는 파일 이름으로 편집 (끝에 ".xlsx"은 필수)
2. Cell을 선택하고 실행 해주세요 (`Shift` + `Enter`).
3. 왼쪽 파일 탐색기에서 엑셀 파일이 만들어집니다. 파일에 오른쪽 마우스 클릭 하시고 `download` 옵션을 선택하세요. 

In [10]:
final.to_excel("파일 이름.xlsx")

#### 프로그램 종료: 브라우저 창을 닫은 후, 저장에 대한 메시지가 나타나면 저장 안 함을 클릭하세요